# Pràctica 2: Recomanador Heurístic

Nom dels alumnes del grup:


**Com fer i estructurar el codi per fer una bona pràctica**: 

+ Definir els paràmetres (input) i el retorn (output) de les funcions de forma clara
+ Definir un ordre per defecte dels usuaris/items. Per exemple, si demanen "el film més ben puntuat", ha de ser el que té un 5 i ID més baix.
+ És MOLT important que la funció que calcula la similitud entre les puntuacions en comú de dos usuaris sigui ràpida!
+ Feu unit-tests de Python

## 1. INTRODUCCIÓ

### 1.1. Abans de començar...

**\+ A més a més de les que ja es troben presents en la 1a cel·la i funcions natives de Python, durant la pràctica, només es podran fer servir les següents llibreries**:

`Pandas, Numpy, Itertools`

**\+ No es poden modificar les definicions de les funcions donades, ni canviar els noms de les variables i paràmetres ja donats**

Això no implica però que els hàgiu de fer servir. És a dir, que la funció tingui un paràmetre anomenat `df` no implica que l'hàgiu de fer servir, si no ho trobeu convenient.

**\+ En les funcions, s'especifica què serà i de quin tipus cada un dels paràmetres, cal respectar-ho**

Per exemple, les funcions tindran [pydoc](https://docs.python.org/3/library/pydoc.html) i allà s'especificarà el paràmetre. `df` sempre serà indicatiu del `Pandas.DataFrame` de les dades.

### 1.2. Dades: puntuacions de pel·licules

La base de dades [movielens-1M](http://www.grouplens.org/node/73) conté 1,000,209 puntuacions de 3.900 pel·lícules fetes l'any 2000 per 6.040 usuaris anònims del recomanador online [MovieLens](http://www.movielens.org/). 

El consum total de tots els usuaris s'hi pot trobar al document "ratings.dat" el format següent:

    UserID::MovieID::Rating::Timestamp

- **UserID** de l'usuari, amb id's entre 1 i 6040 
- **MovieID** de la pel·licula, amb id's entre 1 i 3952
- **Rating** d'un usuari per una pel·licula, en una escala de 1 (menys) a 5 (més) estrelles.
- **Timestamp** que representa quan aquest usuari va puntuar la pel·licula, representat en segons.

La base de dades original està filtrada de manera que cada usuari té com a mínim 20 puntuacions.

### 1.3. Dades: usuaris



Al fitxer ``users.dat`` hi trobem la informació referent a cadascun dels usuaris en el següent format:

        UserID::Gender::Age::Occupation::Zip-code

- **Gender** ve donat per "M" per home i "F" per dona.
- **Age** està representada de la següent forma:

	*  1:  "Under 18"
	* 18:  "18-24"
	* 25:  "25-34"
	* 35:  "35-44"
	* 45:  "45-49"
	* 50:  "50-55"
	* 56:  "56+"

- **Occupation** es tria entre les següents opcions:

	*  0:  "other" or not specified
	*  1:  "academic/educator"
	*  2:  "artist"
	*  3:  "clerical/admin"
	*  4:  "college/grad student"
	*  5:  "customer service"
	*  6:  "doctor/health care"
	*  7:  "executive/managerial"
	*  8:  "farmer"
	*  9:  "homemaker"
	* 10:  "K-12 student"
	* 11:  "lawyer"
	* 12:  "programmer"
	* 13:  "retired"
	* 14:  "sales/marketing"
	* 15:  "scientist"
	* 16:  "self-employed"
	* 17:  "technician/engineer"
	* 18:  "tradesman/craftsman"
	* 19:  "unemployed"
	* 20:  "writer"

Els usuaris han donat la informació voluntariament. Així doncs, la informació d'alguns usuaris pot estar buida.


### 1.4. Dades: pel·lícules



Al fitxer ``movies.dat`` hi trobem la informació referent a cadascuna de les películes en el següent format:

        MovieID::Title::Genres

- **Titles** són identics als titols de la base de dades IMDB, incloent l'any de llançament.
- **Genres** de les películes, que estan separats pel símbol "|" i estan seleccionats d'entre els següents:

	* Action
	* Adventure
	* Animation
	* Children's
	* Comedy
	* Crime
	* Documentary
	* Drama
	* Fantasy
	* Film-Noir
	* Horror
	* Musical
	* Mystery
	* Romance
	* Sci-Fi
	* Thriller
	* War
	* Western

Algunes películes poden tenir l'ID malament degut a duplicats accidentals.

Les películes s'han entrat manualment, així que poden existir altres inconsistencies. 

## 2. Exploració de les dades

### 2.1 Descarregar i llegir dades

+ Baixa't els fitxers que composen la base de dades i els còpies al teu directori de treball. 

In [1]:
# executeu aquesta cel·la per baixar les dades d'internet
# al campus virtual hi ha un fitxer que podeu baixar també
import os
if os.path.isfile("/etc/password.txt") == False:
    os.system('wget -nc http://files.grouplens.org/datasets/movielens/ml-1m.zip')
    os.system('unzip ml-1m.zip')

sh: wget: command not found
unzip:  cannot find or open ml-1m.zip, ml-1m.zip.zip or ml-1m.zip.ZIP.


+ Llegeix les tres taules de la base de dades en tres DataFrames de pandas amb aquest codi:

In [2]:
import math
import numpy as np
import pandas as pd
import datetime
import itertools
from tqdm.notebook import trange, tqdm
import matplotlib.pyplot as plt

In [3]:
unames = ['user_id', 'gender', 'age', 'occupation', 'zip']
users = pd.read_table('ml-1m/users.dat', sep='::', header=None, names=unames, engine='python')
rnames = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_table('ml-1m/ratings.dat', sep='::', header=None, names=rnames, engine='python')
mnames = ['movie_id', 'title', 'genres']
movies = pd.read_table('ml-1m/movies.dat', sep='::', header=None, names=mnames, engine='python', encoding='latin-1')

### 2.2 Inspecció de les taules

In [4]:
users[:5]

,user_id,gender,age,occupation,zip
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [5]:
users[-5:]

,user_id,gender,age,occupation,zip
6035,6036,F,25,15,32603
6036,6037,F,45,1,76006
6037,6038,F,56,1,14706
6038,6039,F,45,0,01060
6039,6040,M,25,6,11106


In [6]:
ratings[-5:]

,user_id,movie_id,rating,timestamp
1000204,6040,1091,1,956716541
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648
1000208,6040,1097,4,956715569


In [7]:
ratings[:5]

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [8]:
ratings.sort_values('movie_id')[:5]

,user_id,movie_id,rating,timestamp
427702,2599,1,4,973796689
1966,18,1,4,978154768
683688,4089,1,5,965428947
596207,3626,1,4,966594018
465902,2873,1,5,972784317


In [9]:
movies[:5]

,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [10]:
ratings[:5]

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


### 2.3 Exemple: Com extreure informació d'un DataFrame.

Suposa que volem calcular les **puntuacions mitjanes d'una pel·licula per sexe o edat**, dades que estan a frames diferents.

El primer pas a obtenir una única estructura que contingui tota la informació. Per fer-ho podem usar la funció ``merge`` de pandas. Aquesta funció infereix automàticament quines columnes ha d'usar per fer el ``merge`` basant-se en els noms que fan intersecció.

Reviseu aquests conceptes de pandas: https://pandas.pydata.org/docs/user_guide/merging.html

In [11]:
data = pd.merge(pd.merge(ratings, users), movies)

# Visualitzem la taula ordenada per identificador d'usuari
data.sort_values(by='user_id')[:10]

,user_id,movie_id,rating,timestamp,gender,age,occupation,zip,title,genres
0,1,1193,5,978300760,F,1,10,48067,One Flew Over the Cuckoo's Nest (1975),Drama
29,1,745,3,978824268,F,1,10,48067,"Close Shave, A (1995)",Animation|Comedy|Thriller
30,1,2294,4,978824291,F,1,10,48067,Antz (1998),Animation|Children's
31,1,3186,4,978300019,F,1,10,48067,"Girl, Interrupted (1999)",Drama
32,1,1566,4,978824330,F,1,10,48067,Hercules (1997),Adventure|Animation|Children's|Comedy|Musical
33,1,588,4,978824268,F,1,10,48067,Aladdin (1992),Animation|Children's|Comedy|Musical
34,1,1907,4,978824330,F,1,10,48067,Mulan (1998),Animation|Children's
35,1,783,4,978824291,F,1,10,48067,"Hunchback of Notre Dame, The (1996)",Animation|Children's|Musical
36,1,1836,5,978300172,F,1,10,48067,"Last Days of Disco, The (1998)",Drama
37,1,1022,5,978300055,F,1,10,48067,Cinderella (1950),Animation|Children's|Musical


In [12]:
data[data['user_id'] == 2]

,user_id,movie_id,rating,timestamp,gender,age,occupation,zip,title,genres
53,2,1357,5,978298709,M,56,16,70072,Shine (1996),Drama|Romance
54,2,3068,4,978299000,M,56,16,70072,"Verdict, The (1982)",Drama
55,2,1537,4,978299620,M,56,16,70072,Shall We Dance? (Shall We Dansu?) (1996),Comedy
56,2,647,3,978299351,M,56,16,70072,Courage Under Fire (1996),Drama|War
57,2,2194,4,978299297,M,56,16,70072,"Untouchables, The (1987)",Action|Crime|Drama
...,...,...,...,...,...,...,...,...,...,...
177,2,356,5,978299686,M,56,16,70072,Forrest Gump (1994),Comedy|Romance|War
178,2,1245,2,978299200,M,56,16,70072,Miller's Crossing (1990),Drama
179,2,1246,5,978299418,M,56,16,70072,Dead Poets Society (1989),Drama
180,2,3893,1,978299535,M,56,16,70072,Nurse Betty (2000),Comedy|Thriller


La funció ``iloc`` ens permet obtenir un subconjunt de files i/o columnes indexades per un enter:

In [13]:
data.iloc[3:5]

,user_id,movie_id,rating,timestamp,gender,age,occupation,zip,title,genres
3,1,3408,4,978300275,F,1,10,48067,Erin Brockovich (2000),Drama
4,1,2355,5,978824291,F,1,10,48067,"Bug's Life, A (1998)",Animation|Children's|Comedy


Els índexs Booleans ens permeten seleccionar una part de la taula que compleix una condició.

In [14]:
# comptem quin tant per cent de ratings estan fets per una dona

print(data[data['gender']=='F']['rating'].count()/float(data['rating'].count())*100, '%')

24.638850480249626 %


Per obtenir les **puntuacions mitjanes de cada pel·licula agrupada per edat** podem usar el mètode ``pivot_table`` que és una forma de "canviar" la forma de la taula especificant quin valor agregat (mitjançant una funció predefinida) hi volem en funció dels valors de dues columnes.

Reviseu aquests conceptes: 
+ https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.pivot.html
+ https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.pivot_table.html#pandas.DataFrame.pivot_table

In [15]:
mean_ratings = data.pivot_table(values= 'rating', index='title', columns='age', aggfunc='mean')
mean_ratings[:10]

age,1,18,25,35,45,50,56
title,,,,,,,
"$1,000,000 Duck (1971)",NaN,3.000000,3.090909,3.133333,2.000000,2.750000,NaN
'Night Mother (1986),2.000000,4.666667,3.423077,2.904762,3.833333,3.555556,4.333333
'Til There Was You (1997),3.500000,2.500000,2.666667,2.900000,2.333333,2.500000,2.666667
"'burbs, The (1989)",4.500000,3.244444,2.652174,2.818182,2.545455,3.208333,2.666667
...And Justice for All (1979),3.000000,3.428571,3.724138,3.657143,4.100000,3.551724,3.928571
1-900 (1994),NaN,NaN,2.000000,NaN,NaN,NaN,3.000000
10 Things I Hate About You (1999),3.745455,3.415020,3.432950,3.102941,3.258065,3.629630,4.000000
101 Dalmatians (1961),3.514286,3.295082,3.613757,3.826087,3.976744,3.650000,3.190476
101 Dalmatians (1996),3.088235,2.467742,2.928571,3.279570,3.482759,3.400000,3.555556


Per obtenir les **puntuacions mitjanes de cada pel·licula agrupada per sexe**:

In [16]:
mean_ratings = data.pivot_table('rating', index='title',columns='gender', aggfunc='mean')
mean_ratings[:10]

gender,F,M
title,,
"$1,000,000 Duck (1971)",3.375000,2.761905
'Night Mother (1986),3.388889,3.352941
'Til There Was You (1997),2.675676,2.733333
"'burbs, The (1989)",2.793478,2.962085
...And Justice for All (1979),3.828571,3.689024
1-900 (1994),2.000000,3.000000
10 Things I Hate About You (1999),3.646552,3.311966
101 Dalmatians (1961),3.791444,3.500000
101 Dalmatians (1996),3.240000,2.911215


Si volgéssim fer càlculs només sobre les pel·licules que han rebut **al menys** 250 puntuacions, primer hem de construir una taula amb el nombre d'avaluacions de cada títol. Per fer-ho, agruparem les dades per títol (amb el mètode ``groupby``) i usarem ``size()``.

Reviseu aquest concepte: 

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.groupby.html

El mètode ``groupby`` implenta un o més d'aquests processos:

+ Dividir les dades segons algun criteri.
+ Aplicar una funció a cada grup.
+ Combinar els resultats en una estructura de dades.

In [17]:
ratings_by_title = data.groupby('title').size()
print(ratings_by_title)

title
$1,000,000 Duck (1971)                         37
'Night Mother (1986)                           70
'Til There Was You (1997)                      52
'burbs, The (1989)                            303
...And Justice for All (1979)                 199
                                             ... 
Zed & Two Noughts, A (1985)                    29
Zero Effect (1998)                            301
Zero Kelvin (Kjærlighetens kjøtere) (1995)      2
Zeus and Roxanne (1997)                        23
eXistenZ (1999)                               410
Length: 3706, dtype: int64


Llavors podem crear un índex amb els títols amb més de 250 avaluacions.

In [18]:
active_titles = ratings_by_title.index[ratings_by_title >= 250]
active_titles

Index([''burbs, The (1989)', '10 Things I Hate About You (1999)',
       '101 Dalmatians (1961)', '101 Dalmatians (1996)', '12 Angry Men (1957)',
       '13th Warrior, The (1999)', '2 Days in the Valley (1996)',
       '20,000 Leagues Under the Sea (1954)', '2001: A Space Odyssey (1968)',
       '2010 (1984)',
       ...
       'X-Men (2000)', 'Year of Living Dangerously (1982)',
       'Yellow Submarine (1968)', 'You've Got Mail (1998)',
       'Young Frankenstein (1974)', 'Young Guns (1988)',
       'Young Guns II (1990)', 'Young Sherlock Holmes (1985)',
       'Zero Effect (1998)', 'eXistenZ (1999)'],
      dtype='object', name='title', length=1216)

L'índex de títols que reben al menys 250 puntuacions es pot fer servir per seleccionar les files de ``mean_ratings``: 

In [19]:
mean_ratings = mean_ratings.loc[active_titles]
mean_ratings

gender,F,M
title,,
"'burbs, The (1989)",2.793478,2.962085
10 Things I Hate About You (1999),3.646552,3.311966
101 Dalmatians (1961),3.791444,3.500000
101 Dalmatians (1996),3.240000,2.911215
12 Angry Men (1957),4.184397,4.328421
...,...,...
Young Guns (1988),3.371795,3.425620
Young Guns II (1990),2.934783,2.904025
Young Sherlock Holmes (1985),3.514706,3.363344


Per veure els films més valorats per les dones, podem ordenar per la columna F de forma descendent:

In [20]:
top_female_ratings = mean_ratings.sort_values(by='F', ascending=False)
top_female_ratings[:10]

gender,F,M
title,,
"Close Shave, A (1995)",4.644444,4.473795
"Wrong Trousers, The (1993)",4.588235,4.478261
Sunset Blvd. (a.k.a. Sunset Boulevard) (1950),4.572650,4.464589
Wallace & Gromit: The Best of Aardman Animation (1996),4.563107,4.385075
Schindler's List (1993),4.562602,4.491415
"Shawshank Redemption, The (1994)",4.539075,4.560625
"Grand Day Out, A (1992)",4.537879,4.293255
To Kill a Mockingbird (1962),4.536667,4.372611
Creature Comforts (1990),4.513889,4.272277


Suposem ara que volem les pel·licules que estan valorades de forma més diferent entre homes i dones. Una forma d'obtenir-ho és afegir una columna a ``mean_ratings`` que contingui la diferència en mitjana i llavors ordenar:

In [21]:
mean_ratings['diff'] = mean_ratings['M'] - mean_ratings['F']

In [22]:
print(np.nan + 9.0) 

nan


Ordenant per ``diff`` ens dóna les pel·licules ben valorades per les dones que presenten més diferència entre homes i dones:

In [23]:
sorted_by_diff = mean_ratings.sort_values(by='diff')
sorted_by_diff[:15]

gender,F,M,diff
title,,,
Dirty Dancing (1987),3.790378,2.959596,-0.830782
Jumpin' Jack Flash (1986),3.254717,2.578358,-0.676359
Grease (1978),3.975265,3.367041,-0.608224
Little Women (1994),3.870588,3.321739,-0.548849
Steel Magnolias (1989),3.901734,3.365957,-0.535777
Anastasia (1997),3.800000,3.281609,-0.518391
"Rocky Horror Picture Show, The (1975)",3.673016,3.160131,-0.512885
"Color Purple, The (1985)",4.158192,3.659341,-0.498851
"Age of Innocence, The (1993)",3.827068,3.339506,-0.487561


Invertint l'ordre de les files i fent un ``slicing`` de les 15 files superiors obtenim les pel·licules ben valorades pels homes que no han agradat a les dones: 

In [24]:
sorted_by_diff[::-1][:15]

gender,F,M,diff
title,,,
"Good, The Bad and The Ugly, The (1966)",3.494949,4.221300,0.726351
"Kentucky Fried Movie, The (1977)",2.878788,3.555147,0.676359
Dumb & Dumber (1994),2.697987,3.336595,0.638608
"Longest Day, The (1962)",3.411765,4.031447,0.619682
"Cable Guy, The (1996)",2.250000,2.863787,0.613787
Evil Dead II (Dead By Dawn) (1987),3.297297,3.909283,0.611985
"Hidden, The (1987)",3.137931,3.745098,0.607167
Rocky III (1982),2.361702,2.943503,0.581801
Caddyshack (1980),3.396135,3.969737,0.573602


Si volguéssim les pel·licules que han generat puntuacions més discordants, independentment del gènere, podem fer servir la variança o la desviació estàndard de les puntuacions: 

In [25]:
rating_std_by_title = data.groupby('title')['rating'].std()

rating_std_by_title = rating_std_by_title.loc[active_titles]
rating_std_by_title.sort_values(ascending=False)[:10]

title
Dumb & Dumber (1994)                     1.321333
Blair Witch Project, The (1999)          1.316368
Natural Born Killers (1994)              1.307198
Tank Girl (1995)                         1.277695
Rocky Horror Picture Show, The (1975)    1.260177
Eyes Wide Shut (1999)                    1.259624
Evita (1996)                             1.253631
Billy Madison (1995)                     1.249970
Fear and Loathing in Las Vegas (1998)    1.246408
Bicentennial Man (1999)                  1.245533
Name: rating, dtype: float64

### Important: Temes de rendiment

Fixeu-vos en el comportament de Python en aquests tres exmepls (que tenen el mateix output). Identifiqueu l'origen de les diferències i actueu en conseqüència:

In [26]:
# Aquesta cel·la pot trigar uns segons a executar-se

%timeit data['title'] 
print(type(data['title']))
%timeit data.title 
print(type(data.title))
%timeit data[['title']] 
print(type(data[['title']]))

722 ns ± 5.27 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)
<class 'pandas.core.series.Series'>
1.42 μs ± 14.2 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)
<class 'pandas.core.series.Series'>
2.56 ms ± 33.2 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)
<class 'pandas.core.frame.DataFrame'>


## 3. EXERCICIS

### 3.1. EXERCICI A

+ Donada la taula ``data`` tal i com es defineix a continuació, calcula la puntuació mitjana de cada usuari i guarda-la a un ``df`` anomenat ``users_mean_rating``. 

In [ ]:
data_folder = 'ml-1m'

In [ ]:
unames = ['user_id', 'gender', 'age', 'occupation', 'zip']
users = pd.read_table(f'{data_folder}/users.dat', sep='::', header=None, names=unames, engine='python')
rnames = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_table(f'{data_folder}/ratings.dat', sep='::', header=None, names=rnames, engine='python')
mnames = ['movie_id', 'title', 'genres']
movies = pd.read_table(f'{data_folder}/movies.dat', sep='::', header=None, names=mnames, engine='python',encoding='latin-1')

data = pd.merge(pd.merge(ratings, users), movies)

# la vostra solució aquí


+ Quina és la pel·lícula més ben puntuada (en mitja) pels usuaris? (Guarda aquest valor en una variable de tipus ``string`` anomenada ``best_movie_rating`` ). Recordeu que  heu de definir un ordre per defecte dels usuaris/items. Si demanen "la pel·lícula més ben puntuada", ha de ser la que té un 5 i ID més baix.

In [ ]:
# la vostra solució aquí


+ Mira si hi ha més pel·licules amb la mateixa puntuació de la més ben puntuada i mostra-les per ordre d'ID.

In [ ]:
# la vostra solució aquí


+ Busca ara aquella pel·lícula, d'entre les que tenen 5 com a puntuació mitjana, que hagi rebut més valoracions i guarda-la a una variable anomenada ``best_movie_rating_maxviews``. Mostra-la. Aixi tindrem la pel·licula més ben puntuada per més usuaris. 


In [ ]:
# la vostra solució aquí


### 3.2. EXERCICI B

+ Defineix una funció anomenada ``top_movie`` que donat un usuari ens retorni quina és la pel·lícula millor puntuada i ID més baix..


In [ ]:
def top_movie(dataFrame,usr):
    # la vostra solució aquí

print(top_movie(data,1))

### 3.3. EXERCICI C

+ Construeix una funció que donat el dataframe ``data`` et retorni un altre dataframe ``df_counts``amb el valor que cada usuari li ha donat a una peli. Això ho farem creant un dataframe on les columnes estiguin indexades per `movie_id`, les files per `user_id` i els valors siguin el rating donat.

In [ ]:
def build_counts_table(df):
    """
    Retorna un dataframe on les columnes són els `movie_id`, les files `user_id` i els valors
    la valoració que un usuari ha donat a una peli d'un `movie_id`
    
    :param df: DataFrame original 
    :return: DataFrame descrit adalt
    """
    
    # la vostra solució aquí


In [ ]:
df_counts = build_counts_table(data)
df_counts

+ Fés una funció `get_count` que donada la taula anterior i dos id's (usuari i peli), extregui el valor donat:

In [ ]:
def get_count(df, user_id, movie_id):
    """
    Retorna la valoració que l'usuari 'user_id' ha donat de 'movie_id'
    
    :param df: DataFrame retornat per `build_counts_table`
    :param user_id: ID de l'usuari
    :param movie_id: ID de la peli
    :return: Enter amb la valoració de la peli
    """
    
    # la vostra solució aquí


get_count(df_counts, 1, 1)

### 3.4. EXERCICI D

In [ ]:
data.nunique()

In [ ]:
unique_movies = pd.unique(data['movie_id'])
unique_movies.max()

Si observem el nombre total d'usuaris únics i de pel.licules úniques, podem veure que els id's dels usuaris van de 1 a 6040. Normalment volem índexos que comencin al nombre 0, anant de 0 a 6039. 

+ Explora els índexos de les pel·licules. **Quin problema hi ha amb els indexos de les pel·licules??**

> **Resposta**  


+ Usant la funció `pd.Categorical(*).codes`, re-indexa els id's dels usuaris i de les pelis perquè vagin de 0 a 6039 i de 0 a 3705 respectivament:

In [ ]:
# la vostra solució aquí


In [ ]:
data[data['user_id'] == 2]

+ Per comprovar que tot sigui correcte i guardar correctament la taula **df_counts**, torna a calcular i visualitza ``df_counts``:

In [ ]:
df_counts = build_counts_table(data)
df_counts

### 3.5. EXERCICI E



+ Escriu una funció `distEuclid(x,y)`  que implementi la **distància** Euclidiana entre dos vectors usant funcions de numpy. 

+ Escriu la funció `simEuclid(U1, U2)` que calculi la **similitud** entre dos vectors segons la fòrmula següent (on $n$ és un factor de normalització). Ho fem així perquè si dos usuaris tenen moltes pel·licules en comú, volem que la similitud entre aquests usuaris sigui major que el de dos usuaris que només n'han vist una en comú. Si els vectors estan buits, retornar 0.

    $$d =  \frac{1}{(1+distEuclid(U1, U2))} \times \frac{len(U1)}{n} $$

+ Escriu la funció `simUsuaris(df, U1, U2)` per retorna la sembalça de dos usuaris a partir del `df_counts`, tenint en compte les puntuacions que tenen en comú, fent servir les dues funcions anteriors.
    
+ Avalueu amb la funció ``%timeit`` quant triguen aquests càlculs per un parell d'usuaris.   

> *Nota: Alguns d'aquests exercicis tenen temps de càlcul de l'ordre de minuts sobre tota la base de dades. Per desenvolupar els algorismes és recomanable treballar amb una versió reduïda de la base de dades.* 

Per implementar aquestes funcions únicament es permet l'ús de les funcions:

* `np.sum`
* `np.sqrt`
* `np.power`
* `np.dot`
* `np.linalg.norm`
* `np.mean`

I s'ha de fer **sense bucles**!

In [ ]:
num_movies = data.nunique()['movie_id']

def distEuclid(x, y):
    """
    Retorna la distancia euclidiana de dos vectors n-dimensionals.
    
    :param x: Primer vector
    :param y: Segon vector
    :return : Escalar (float) corresponent a la distancia euclidiana
    """
    
    # la vostra solució aquí


def simEuclid(Vec1, Vec2, norm):
    """
    Retorna la sembalça de dos vectors.
    
    :param Vec1: Primer vector
    :param Vec2: Segon vector
    :return : Escalar (float) corresponent a la semblança
    """
    # la vostra solució aquí


def simUsuaris(DataFrame, User1, User2):
    """
    Retorna un score que representa la similitud entre user1 i user2 basada en la distancia euclidiana
    
    :param DataFrame: dataframe que conté totes les dades
    :param User1: id user1
    :param User2: id user2
    :return : Escalar (float) corresponent al score
    """
    
    # la vostra solució aquí
    

In [ ]:
print(simUsuaris(df_counts, 2,314))

In [ ]:
# Aqui ha de srotir un valor de l'ordre de microsegons, no milisegons

%timeit simUsuaris(df_counts, 1, 5)

### 3.6. EXERCICI F

En aquest exercici desenvoluparem un sistema de recomanació col·laboratiu **basat en usuaris**. 

La funció principal, ``getRecommendationsUser``, ha de tenir com a entrada una taula de puntuacions, un ``user_id``, el tipus de mesura de similitud (Euclidiana) que volem usar, el nombre `m` d'usuaris semblants que volem per fer la recomanació i el nombre ``n`` de recomanacions que volem. 

Exemple: ``getRecommendationsUser(data, 2, 50, 10, simEuclid)``

Com a sortida ha de donar la llista de les $n$ millors pel·lícules que li podriem recomanar segons la seva semblança amb altres usuaris.

> *Nota 1: S'ha d'evitar comparar ``user_id`` a ell mateix.*

> *Nota 2: Recordeu que en Python podem passar funcions com a paràmetres d'una funció.*

#### EXERCICI F.1

+ Computa la *score* de similitud del usuari desitjat (``userID``) respecte tots els altres i retorna un diccionari dels $m$ usuaris més propers i el seu *score* de semblança. Fes servir la matriu `df_counts` Normalitzeu els *scores* de sortida de manera que sumin 1.

In [ ]:
def find_similar_users(DataFrame, userID, m, simfunction):
    """
    Retorna un diccionari de usuaris similars amb les scores corresponents.
    
    :param DataFrame: dataframe que conté totes les dades
    :param userID: usuari respecte al qual fem la recomanació
    :param m: nombre d'usuaris que volem per fer la recomanació
    :param similarity: mesura de similitud
    :return : dictionary
    """
    # la vostra solució aquí
    

In [ ]:
t = datetime.datetime.now()
sim_dict = find_similar_users(df_counts, 2, 10, simUsuaris)
t = datetime.datetime.now()-t
print(str(t))

In [ ]:
sim_dict

+ Quan trigaria (en minuts) si ho fem per tots els usuaris?

In [ ]:
# la vostra solució aquí


+ Anem ara a construir una `matriu` de mida $U \times U$ on cada posició $(i,j)$ indiqui la distància entre l'element $i$ i el $j$. Així doncs, si estàs fent un recomanador basat en usuaris, `matriu[2, 3]` contindrà la similitud entre l'usuari 2 i el 3.

Compareu aquestes dues opcions des del punt de vista de temps de càlcul:

* Feu una funció,  que construeixi la ``similarity_matrix1`` a partir de la distancia entre usuaris com la distància enre els vectors formats pels elements en comú dels dos usuaris.
* Feu una funció que construeixi la ``similarity_matrix2`` d'una forma *aproximada*:
    + Substituïnt els ``nand`` que corresponen a ítems no avaluats per ``0``. 
    + Treballant específicament amb operacions matricials. En aquest link podeu trobar indicacions de com fer-ho: https://jaykmody.com/blog/distance-matrices-with-numpy/

In [ ]:
def compute_similitude(fixed_arr, var_arr):
    """
    Donats dos vectors, calcula la similitud entre els subvectors formats 
    pels elements en comú (sense fer servir cap iteració!). 
    Normalitzeu la sortida multiplicant pel nombre de pel·lícules vistes en comú i
    dividint pel nombre total de pelis del dataset
    """
    
    # la vostra solució aquí


In [ ]:
# test

vec1 = np.array([1, np.nan, 2, 3, 4])
vec2 = np.array([np.nan, 4, 5, 2, 2])
print(compute_similitude(vec1, vec2))

vec1 = np.array([1, np.nan, 1, 1, 1])
vec2 = np.array([np.nan, 1, 1, 1, 1])
print(compute_similitude(vec1, vec2))

vec1 = np.array([1, 1, 1, 1, 1])
vec2 = np.array([1, 1, 1, 1, 1])
print(compute_similitude(vec1, vec2)*num_movies/5)

In [ ]:
def similarity_matrix_1(compute_distance, df_counts):
    """
    Retorna una matriu de mida M x M on cada posició 
    indica la similitud entre usuaris (resp. ítems).
    
    :param df_counts: df amb els valor que cada usuari li ha donat a una peli.
    :return : Matriu numpy de mida M x M amb les similituds.
    """
    
    # la vostra solució aquí


In [ ]:
t = datetime.datetime.now()
sim = similarity_matrix_1(compute_similitude, df_counts)
t = datetime.datetime.now()-t
print("Temps amb doble for:",str(t))

In [ ]:
def similarity_matrix_2(DataFrame):
    """
    Retorna una matriu de mida M x M on cada posició 
    indica la similitud entre usuaris (resp. ítems).
    Substitueix els nand per 0.

    :return : Matriu numpy de mida M x M amb les similituds.
    """
    # la vostra solució aquí


In [ ]:
t = datetime.datetime.now()
sim = similarity_matrix_2(df_counts)
t = datetime.datetime.now()-t
print("Temps matricialment:",str(t))

In [ ]:
sim

+ Ara torna a re-fer la funció ``find_similar_users`` usant la matriu de distàncies i mira quant triga. Recorda que les scores han d'estar normalitzades!

In [ ]:
def find_similar_users(DataFrame, sim_mx, userID, m):
    # la vostra solució aquí


In [ ]:
t = datetime.datetime.now()
sim_dict = find_similar_users(df_counts, sim, 2, 10)
t = datetime.datetime.now()-t
print(str(t))

In [ ]:
sim_dict

#### EXERCICI F.2

+ Computa les recomanacions per un usuari concret a partir dels scores dels seus $m$ usuaris més propers. 
    + Fes primer una funció ``weighted_average`` que retorni un diccionari del tipus ``{peli_id: score predit}`` amb la puntuació predita de cada ítem a partir de les puntuacions dels $m$ usuaris més propers i de la seva semblança a l'usuari considerat.
    + Fes després una funció ``getRecommendationsUser`` que usant la funció anterior retorni un ``df`` amb els $n$ ítems amb més score i els seus scores.

In [ ]:
def weighted_average(DataFrame, user, sim_mx, m):
    """    
    :param DataFrame: dataframe que conté totes les dades
    :param user: usuari al qual fem la recomanació
    :param sim_mx: similarity_matrix
    :param m: nombre d'usuaris semblants a tenir en compte per les recomanacions
    :return: diccionari {peli_id: score predit}
    """
    # la vostra solució aquí
      


In [ ]:
def getRecommendationsUser(DataFrame, user, sim_mx, n, m):
    """    
    :param DataFrame: dataframe que conté totes les dades
    :param user: usuari al qual fem la recomanació
    :param sim_mx: similarity_function
    :param n: nombre de pelis a recomanar
    :param m: nombre d'usuaris semblants a tenir en compte per les recomanacions
    :return : dataframe de pel·licules amb els scores.
    """
    
    # la vostra solució aquí
    


In [ ]:
t = datetime.datetime.now()
user_prediction = getRecommendationsUser(data, 3, sim, 10, 50)
t = datetime.datetime.now()-t
print(str(t))

In [ ]:
user_prediction

### 3.7. EXERCICI G


A continuació usarem la metrica **Mean Absolute Error (MAE)** per evaluar el nostre sistema. Aquesta mètrica ens permetrà mesurar la diferencia entre dues llistes donat un usuari: 
+ La llista amb els scores reals d'un usuari
+ La llista amb els scores predits per aquest usuari

#### EXERCICI G.1

Anem a crear un conjunt de training i un de test de forma "ingènua":
+ Selecciona de forma aleatòria el 10% dels usuaris i guarda'ls en una llista anomenada ``test_set``.  
+ Guarda la resta en una llista anomenada ``train_set``.
+ Mira quants elements tenen aquestes llistes.

In [ ]:
# la vostra solució aquí


+ Què passarà si calculo la matriu de similitud amb ``train_set`` i després intento predir pels usuaris de ``test_set``??

> **Resposta**



#### EXERCICI G.2

Cambien ara la manera de generar els conjunts per no tenir el problema anterior.

+ Seleccionarem aproximadament el 80% de les interaccions de cada usuari de ``test_set`` i les afegirem al ``train_set``. 
+ Podriem ara podem evaluar el sistema?

> Us donem el codi per un usuari donat i vosaltres només heu de crear la funció que, per cada usuari, afageixi el 80% de les intraccions al ``train_set``.

In [ ]:
test_set.head()

In [ ]:
# Agafem el 20% de les pelis que ha consumit cada usuari de test 
groupby_count = test_set.groupby('user_id')['movie_id'].count()*0.2
groupby_count

Seleccionem la posició 1 i aquest ``user_id`` serà el que usarem pel codi d'exemple (que després haureu de replicar).

In [ ]:
groupby_count.reset_index().iloc[1]

In [ ]:
n_test_samples = int(groupby_count.reset_index().iloc[1]['movie_id'])
u = groupby_count.reset_index().iloc[1]['user_id']

In [ ]:
test_set_user = test_set[test_set['user_id'] == u]
frame_test = test_set_user.sample(n_test_samples)
print("TOTAL SAMPLES OF THE USER: " + str(len(test_set_user)))
print("TOTAL SAMPLES OF THE USER IN TEST SET: " + str(len(frame_test)))

In [ ]:
len(test_set_user.index)

In [ ]:
frame_train = test_set_user[~test_set_user.index.isin(frame_test.index)]
print("TOTAL SAMPLES OF THE USER IN TRAIN SET: " + str(len(frame_train)))

In [ ]:
assert len(frame_train) + len(frame_test) == len(test_set_user)

In [ ]:
def add_testdata(traindf, test_set):
    """    
    :param traindf: dataframe que conté les dades de train
    :param test_set: dataframe que conté les dades de test

    :return : 
        - :param 1st: dataframe que conté les dades de train juntament amb el 80% de test seleccionat
        - :param 2nd: dataframe que conté les dades de test que queden (20% restant)
    """
    
    # la vostra solució aquí
    

In [ ]:
train, test = add_testdata(train_set, test_set)

In [ ]:
train

In [ ]:
test

In [ ]:
train.shape

In [ ]:
test.shape

In [ ]:
data.shape

In [ ]:
assert train.shape[0] + test.shape[0] == data.shape[0]

#### EXERCICI G.3

+ Fes una funció que serveixi per evaluar el nostre sistema usant la mètrica MAE. 

In [ ]:
def evaluateRecommendations(train, test, m,n, sim):
    """
    Retorna l'error generat pel model
    
    :param DataFrame: dataframe que conté totes les dades
    :param userID: usuari respecte al qual fem la recomanació
    :param m: nombre d'usuaris que volem per fer la recomanació
    :param n: nombre de pelis a retornar (no)
    :param sim: matriu de similitud
    :return : Escalar (float) corresponent al MAE
    """
   
    # la vostra solució 
    


In [ ]:
t = datetime.datetime.now()
mae = evaluateRecommendations(train, test, 50, 10, sim)
t = datetime.datetime.now()-t
print(str(t))

In [ ]:
mae

### 3.8. EXERCICI H (exercici opcional, no obligatori)


+ **Que surt més a compte, fer un recomanador unic pels dos sexes o un per cada sexe?** Justifica la resposta per escrit i amb el codi necessari.

In [ ]:
# la vostra solució aquí
